In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.optimize import minimize, differential_evolution

In [66]:
#加载数据
df = pd.read_excel('各参数SEA.xlsx')
# print(df.sample(10))

In [67]:
#描述性统计
print(df.describe())
print(sea_range := df['SEA'].max() - df['SEA'].min())

             α(°)       β(°)       l1/l2        t_mm          SEA
count  560.000000  560.00000  560.000000  560.000000   560.000000
mean    45.000000   55.00000    2.250000    1.150000   669.679696
std     10.008941    7.07739    0.559517    0.111903   151.469724
min     30.000000   45.00000    1.500000    1.000000   368.340000
25%     35.000000   50.00000    1.875000    1.075000   544.565000
50%     45.000000   55.00000    2.250000    1.150000   684.160000
75%     55.000000   60.00000    2.625000    1.225000   785.387500
max     60.000000   65.00000    3.000000    1.300000  1023.070000
654.73


In [68]:
#数据拆分
X_train, X_test, y_train, y_test = train_test_split(
    np.array(df[['α(°)',  'β(°)',  'l1/l2',  't_mm',]]),
    np.array(df[['SEA']]),
    test_size=0.2,
    shuffle=True,
    random_state=42
)

In [69]:
#训练梯度提升树模型
model = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=20,
    min_samples_leaf=10,
    subsample=0.8,
    random_state=42,
)
model.fit(X_train, y_train.ravel())

,loss,'squared_error'
,learning_rate,0.1
,n_estimators,100
,subsample,0.8
,criterion,'friedman_mse'
,min_samples_split,20
,min_samples_leaf,10
,min_weight_fraction_leaf,0.0
,max_depth,5
,min_impurity_decrease,0.0
,init,None


In [70]:
#模型评估
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2_test = r2_score(y_test, y_pred)
r2_train = r2_score(y_train, model.predict(X_train))
print(rmse)
print(r2_test)
print(r2_train)

20.808039175167856
0.9823049971042702
0.9899993007086643


In [71]:
#优化
##定义目标函数
def objective(x):
    x = np.array(x).reshape(1, -1)
    y = model.predict(x)
    return -y

In [72]:
##定义边界约束
param_bounds = []
for i in df.iloc[:, 0:4]:
    param_bounds.append((df[i].min(), df[i].max()))
# print(param_bounds)

In [73]:
##minimize
times = 20
best_result = None
for i in range(times):
    if i < times:
        x0 = np.array([np.random.uniform(low, high) for low, high in param_bounds])
    else:
        x0 = np.array(df.iloc[:, 0:4].median())
    result = minimize(
                objective,
                x0,
                bounds=param_bounds,
                method='SLSQP',
                options=dict(
                    disp=False,
                    maxiter=100,
                    )
                )
    print(f'第{i+1}次起始点{x0}结果{-result.fun}')
    if best_result is None or result.fun < best_result.fun:
        best_result = result
optimal_params = best_result.x
optimal_sea = -best_result.fun
print(optimal_params)
print(optimal_sea)

第1次起始点[42.25130351 62.57335116  2.02072917  1.19673723]结果844.8758320373935
第2次起始点[38.73092709 57.64284427  1.85439572  1.16065686]结果866.3074199298181
第3次起始点[43.90265143 51.17363515  2.44491107  1.23705432]结果723.8255802385979
第4次起始点[54.42667474 57.1647054   1.83620855  1.09738641]结果614.9619579550382
第5次起始点[50.02138273 51.58710634  1.76408013  1.14567488]结果614.5569621142376
第6次起始点[54.51880445 54.60670616  2.20241303  1.20787358]结果716.9190884865964
第7次起始点[34.74737729 58.49094454  2.16409481  1.09709117]结果785.4186398826172
第8次起始点[51.1124537  50.74251704  1.97537775  1.19768231]结果713.687121304063
第9次起始点[49.56988243 62.88586994  1.59370009  1.03484421]结果459.45530488121807
第10次起始点[43.14877941 47.04246905  2.57739709  1.05139225]结果611.6058578267289
第11次起始点[31.93158272 49.96795489  1.75106215  1.23337522]结果825.6707383473434
第12次起始点[32.34132748 51.01112807  1.52788699  1.13628769]结果723.2209236655186
第13次起始点[51.79504245 55.52742574  2.25058591  1.20303248]结果704.5169066899033
第14次起始点[58.65165145 6